# Script to automate the export and manipulation of the VICAV-library

## Import Package eTree to parse XML Files

In [1]:
import requests
import json
import logging
import os
import re
import xml
import xml.etree.ElementTree as ET
import asyncio
import aiohttp
# this module is needed to make asyncio.run work inside the notebook as well as in the generated python script
import nest_asyncio
nest_asyncio.apply()
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
#logging.basicConfig(level=logging.DEBUG)

## Define name-space for xml-parsing

In [2]:
xmlns = {"tei": "http://www.tei-c.org/ns/1.0", "xml":"http://www.w3.org/XML/1998/namespace" }

## Access to the VICAV Zotero library

* Use API_TOKEN from environment to access Zotero
* Set the Zotero group id for VICAV here

In [3]:
request_headers = {'Authorization': 'Bearer ' + os.environ['API_TOKEN']}
group_id = "2165756"
limit_downloads_to = None
conn_limit=16
total_timeout=600 #s

## Read all items in the library

Load items from Zotero group library

    Args: 
        group_id (str): ID of a Zotero group
        limit (int): number of items to retrieve from library, maximum is 100.
        start (int): item number to start with

In [4]:
async def get_items(session, group_id:str,limit:int,start:int,itemType = None):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/" + "?limit=" + str(limit) + "&start=" + str(start) + ("&itemType="+itemType if itemType is not None else "")
    async with session.get(url=request_url, headers=request_headers) as response:
        if response.status == 200:
            parsed = json.loads(await response.text())
            response_headers = response.headers
        
    return parsed, response_headers

Get total number of items in group library

    Args:  
        group_id (str): ID of a Zotero group
    
    Returns:
        int: number of items in the library

In [5]:
def total_number_items(group_id) -> int:
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/"
    response = requests.get(request_url, headers=request_headers)
    
    return int(response.headers["Total-Results"])

Get headers of Zotero-Api-Calls

    Args:  
        group_id (str): ID of a Zotero group

In [6]:
def get_headers(group_id):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/"
    response = requests.get(request_url, headers=request_headers)
    
    return response.headers

Get links from headers

    Args:
        headers: http-headers of a response

    Returns:
        dict

In [7]:
def get_links_from_headers(headers) -> dict:
    link_list = headers["Link"].split(",")
    links = {}
    for link_item in link_list:
        #print(link_item)
        link_type = link_item.split('; rel="')[1].replace('"','').strip()
        link_value = link_item.split('; rel="')[0].replace("<","").replace(">","").strip()
        links[link_type] = link_value
    
    return links

Get all items of a collection

TODO:
* Generate all links with `for start in range(limit,last,limit)`.
* Then download in parallel.

In [8]:
async def get_all_items(session, group_id, itemType = None):
    logging.info("Getting all items" + ((" of type " + itemType) if itemType is not None else "") + " now.")
    # empty list that will hold all items of the library
    allitems=[]
    
    # settings to be used in the function to get the items (limit is max 100)
    limit=100
    start=0
    
    # get the first 200 items to start with
    first_round=await get_items(session, group_id,limit,start,itemType)
    allitems=allitems+first_round[0]
    
    # get the next link from the headers
    next_url = get_links_from_headers(first_round[1])["next"]
    last_url = get_links_from_headers(first_round[1])["last"]
    # get items until next url is last url, then all items are fetched
    while next_url != last_url:
        logging.info("Getting items from " + next_url)
        async with session.get(url=next_url, headers=request_headers) as response:
            if response.status == 200:
                parsed = json.loads(await response.text())
                response_headers = response.headers

                allitems=allitems + parsed
                urls = get_links_from_headers(response_headers)
                if "next" in urls:
                    next_url = urls["next"]
                else:
                    break
            else:
                break

    # get the last items of the group
    response = requests.get(last_url)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        allitems=allitems + parsed
    
    return allitems

Store all items of a group library in a json file

    Args:
        group_id (str): ID of a Zotero group
        filename (str): name of the export file including file-extension

    Returns:
        bool: True if successful

In [9]:
def export_all_items_to_file(group_id,filename) ->bool: 
    allitems = get_all_items(group_id)
    with open(filename,"w") as f:
        json.dump(allitems, f)
    return True

Store export in a file and get all item ids

The export contains also the note items. These are child items of some other item in this export. They have a parent reference.

There are also attachment items. These are child items of some other item in this export. Most of them have a parent reference but some don't have a parent item (anymore?)

In [10]:
json_file = "export_grouplib.json"
item_ids = []
note_ids = []
attachment_ids = []
async def get_generic_items(session):
    if os.path.isfile(json_file):
        logging.info("Grouplib export json already exists. Delete to fetch again (time consuming).")
        with open(json_file, 'r') as f:
            all_items = json.load(f)    
    else:
        all_items = await get_all_items(session, group_id)
    return all_items

async def get_export_json():
    conn = aiohttp.TCPConnector(limit=conn_limit)
    timeout = aiohttp.ClientTimeout(total=total_timeout)
    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        all_items = await get_generic_items(session)
    # all_items = test5
    with open(json_file,"w") as f:
        json.dump(all_items, f)
        logging.info("Exported json.")
    
    for item in all_items:
        item_id = item["key"]
        item_type = item["data"]["itemType"]
        if item_type == 'note':
            note_ids.append(item_id)
        elif item_type == 'attachment':
            attachment_ids.append(item_id)
        else:
            item_ids.append(item_id)
    return all_items
all_items = asyncio.run(get_export_json())
all_items_map = {data["key"]:data for data in all_items}
all_notes_map = {data["data"]["parentItem"]:data for data in [all_items_map[id] for id in note_ids]}
all_attachments_map = {data["data"]["parentItem"] if "parentItem" in data["data"] else "ZZZZZZZZ":data for data in [all_items_map[id] for id in attachment_ids]}

2023-03-01 16:04:00,133 - Getting all items now.
2023-03-01 16:04:09,057 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=100
2023-03-01 16:04:14,464 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=200
2023-03-01 16:04:19,279 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=300
2023-03-01 16:04:24,055 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=400
2023-03-01 16:04:29,209 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=500
2023-03-01 16:04:33,920 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=600
2023-03-01 16:04:38,376 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=700
2023-03-01 16:04:43,993 - Getting items from https://api.zotero.org/groups/2165756/items/?limit=100&start=800
2023-03-01 16:04:48,820 - Getting items from https://api.zotero.org/gro

## Get the geo data and replace tags with geo refs

geo data is in `../../vicav_biblio/vicav_geodata.xml`

In [12]:
geo_data = ET.parse("../../010_manannot/vicav_geodata.xml")
geo_parent_map = {c:p for p in geo_data.iter( ) for c in p}
place_by_name = {placeName.text: 
                 {"type": geo_parent_map[placeName].get("type"),
                  "geo": geo_parent_map[placeName].find("./tei:location/tei:geo[@decls='#dd']",xmlns).text,
                  "el": geo_parent_map[placeName]}
                for placeName in geo_data.findall(".//tei:listPlace/tei:place/tei:placeName", xmlns)}

# Get the current mapping for xml:id to Zoteros unique ID

Zotero suggests readable @xml:id but those are not unique between runs or  
take into account that there may be more than one works by one author in a year.

In [14]:
get_zotero_unique_id = re.compile(r'https?://zotero.org/groups/[\d]+/items/(?P<zuid>[A-Z0-9]+)')
current_bibl_data = ET.parse("../../010_manannot/vicav_biblio_tei_zotero.xml")
xmlid_by_zuid = {get_zotero_unique_id.match(bibStr.get("corresp")).groupdict()["zuid"]: bibStr.get("{http://www.w3.org/XML/1998/namespace}id")
                 for bibStr in current_bibl_data.findall(".//tei:biblStruct", xmlns)}

## Get all TEIs from Zotero

man nimmt die Liste mit den IDs der entries, baut für jeden entry die URL nach dem Muster  
https://api.zotero.org/groups/2165756/items/944KQVKQ?format=tei  
man lädt das mit GET requesst  
dann aus dem response den body und parsed das mit ET from string, nimmt daraus das  
`<biblStruct>` Element;  
baut eine gemeinsame `<listBibl>` und fügt das geparste Element ein,  
dann dumpt man den ganzen Element-Tree

### Retrieves TEI of an item generated by Zotero

Resolves place names to geo coordinates using the place by name dict created above.

In [15]:
ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
tag_parser = re.compile(r'(?P<geo_type>[^:]+):(?P<geo_name>[^[]+)(\[(?P<long>[\d.,]+) +(?P<lat>[\d.,]+)])?')

def create_geo_tag(tags_el, tag):
    #starts with reg: geo: diaGroup: -> lookup, get geo location and create elemnt
    # uses PEP 634, requires python 3.10+
    m = tag_parser.match(tag)
    if m is not None:
        match m.groupdict():
            case {"geo_type": "reg" | "geo" | "diaGroup", "geo_name": geo_name, "long": long, "lat": lat}:
                tag_note_el = ET.SubElement(tags_el, "note", type="tag")
                geo_name = geo_name.rstrip()
                ET.SubElement(tag_note_el, "name", type=m.groupdict()["geo_type"]).text = geo_name
                if geo_name in place_by_name:
                    ET.SubElement(tag_note_el, "geo").text = place_by_name[geo_name]["geo"]
                else:
                    ET.SubElement(tag_note_el, "note", type="missing_geo_data")
                return tag_note_el
    if tag in place_by_name:
        tag_note_el = ET.SubElement(tags_el, "note", type="tag", subtype="unmarked_geo")
        ET.SubElement(tag_note_el, "name", type=place_by_name[tag]["type"]).text = tag
        ET.SubElement(tag_note_el, "geo").text = place_by_name[tag]["geo"]
        return tag_note_el
    ret = ET.SubElement(tags_el, "note", type="tag")
    ret.text = tag
    return ret

async def get_item_tei(group_id,item_id,session):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/" + item_id + "?format=tei"
    bibl = None
    note_for_item_id = ""
    response_text = ""
    try:
        async with session.get(url=request_url, headers=request_headers) as response:
            response_text = await response.text()
        list_bibl = ET.fromstring(response_text)
        bibl = list_bibl.find("tei:biblStruct",xmlns)
        if bibl is None:
            logging.info("There is no biblStruct in the response for item " + item_id +":\n"+response_text)
            return bibl
        zuid = get_zotero_unique_id.match(bibl.get("corresp")).groupdict()["zuid"]
        current_xmlid = bibl.get("{http://www.w3.org/XML/1998/namespace}id")
        if zuid in xmlid_by_zuid and current_xmlid != xmlid_by_zuid[zuid]:
            bibl.set("{http://www.w3.org/XML/1998/namespace}id", xmlid_by_zuid[zuid])
            logging.info("Changed @xml:id for item " + item_id + " from "+ current_xmlid + " to " + xmlid_by_zuid[zuid] + ".")
        if item_id in all_notes_map:
            note_for_item_id = all_notes_map[item_id]["data"]["note"].replace("&", "&amp;")
            parsed_note = ET.fromstring("<note>"+note_for_item_id+"</note>")
            bibl.append(parsed_note)
        tags = all_items_map[item_id]["data"]["tags"]
        if len(tags) > 0:
            tags_el = ET.SubElement(bibl, "note", type="tags")
            for o in tags:
                create_geo_tag(tags_el, o["tag"])
                
    except asyncio.TimeoutError:
        logging.info("Timeout fetching " + item_id)
    except xml.etree.ElementTree.ParseError:
        logging.info("XML parser error in notes for item id "+item_id+"\n"+note_for_item_id)
    if bibl is None:
        logging.debug("No biblStruct in item " + item_id)
    logging.info("Fetched TEI for " + item_id)
    return bibl

In [18]:
async def get_item_tei_test():
    conn = aiohttp.TCPConnector(limit=conn_limit)
    timeout = aiohttp.ClientTimeout(total=total_timeout)
    async with aiohttp.ClientSession(connector=conn) as session:
        #test = await get_item_tei(group_id,"944KQVKQ",session)
        #test = await get_item_tei(group_id,"DXNCFAMR",session)
        #test = await get_item_tei(group_id,"6QNLQCG9",session)
        test = await get_item_tei(group_id,"D25ZTU9X",session)
    ET.indent(test)
    ET.dump(test)
asyncio.run(get_item_tei_test())

2023-03-01 16:14:46,691 - Fetched TEI for D25ZTU9X


<biblStruct xmlns="http://www.tei-c.org/ns/1.0" type="bookSection" xml:id="Al-Essa_Aziza2022" corresp="http://zotero.org/groups/2165756/items/D25ZTU9X">
  <analytic>
    <title level="a">Arabic Interdentals: Variation and Linguistic Change</title>
    <author>
      <name>Al-Essa, Aziza</name>
    </author>
  </analytic>
  <monogr>
    <title level="m">Semitic dialects and dialectology : fieldwork - community - change</title>
    <idno type="ISBN">978-3-96822-095-6 / Druck:  9783968220963</idno>
    <editor>
      <name>Klimiuk, Maciej</name>
    </editor>
    <imprint>
      <pubPlace>Heidelberg</pubPlace>
      <biblScope unit="page">45-63</biblScope>
      <publisher>Heidelberg University Publishing</publisher>
      <date>2022</date>
    </imprint>
  </monogr>
</biblStruct>


In [19]:
template = ET.parse("listbibl_template.xml")
list_bibl = template.find("tei:text/tei:body/tei:listBibl",xmlns)

### Load template containing a listBibl-element that will be filled with the retrieved biblStruct elements

## Get the TEI

* For each item-id get the TEI and append it to list-bibl
* Save the resulting XML
* Save errors for further inspection

We need to consider https://www.zotero.org/support/dev/web_api/v3/basics#rate_limiting

TODO: Get the all the top items in the group with https://api.zotero.org/groups/2165756/items/top?format=tei&limit=100&sort=creator
* Generate all links with `for start in range(limit,last,limit)`.
* Then download in parallel.

In [20]:
errors = []
max_fetch = limit_downloads_to if limit_downloads_to is not None else len(item_ids)
logging.info("Fetching at most " + str(max_fetch) + " TEI/XML.")
async def get_item_tei_from_group(item_id, session):
    bibl_struct = await get_item_tei(group_id, item_id, session)
    if bibl_struct:
        list_bibl.append(bibl_struct)
    else:
        logging.debug("Can not append " + item_id)
        errors.append(item_id)

async def async_download():
    conn = aiohttp.TCPConnector(limit=conn_limit)
    timeout = aiohttp.ClientTimeout(total=total_timeout)
    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        await asyncio.gather(*[get_item_tei_from_group(item_id, session) for item_id in item_ids[:max_fetch]])

asyncio.run(async_download())

list_bibl[:] = sorted(list_bibl, key=lambda child: child.get("{http://www.w3.org/XML/1998/namespace}id"))
with open('TEI_export.xml', 'wb') as f:
    ET.indent(template)
    template.write(f, encoding='utf-8',xml_declaration=True)
    logging.info("TEI export done.")

# Export IDs of items with errors
with open("errors.json","w") as f:
    json.dump(errors, f)
    logging.info("Exported errors.json.")

2023-03-01 16:15:03,561 - Fetching at most 5153 TEI/XML.
2023-03-01 16:15:07,731 - Changed @xml:id for item 6EPNLRXU from Retsö2009 to Retsö2009b.
2023-03-01 16:15:07,732 - Fetched TEI for 6EPNLRXU
2023-03-01 16:15:07,752 - Fetched TEI for ZN8W8HVW
2023-03-01 16:15:07,757 - Fetched TEI for JI8T4NTP
2023-03-01 16:15:07,840 - Fetched TEI for UM84SU4X
2023-03-01 16:15:07,884 - Fetched TEI for 4ULBE9CY
2023-03-01 16:15:07,890 - Changed @xml:id for item NPJ679RJ from Arnold2015 to Arnold2015a.
2023-03-01 16:15:07,894 - Fetched TEI for NPJ679RJ
2023-03-01 16:15:07,907 - Changed @xml:id for item B6L9AIMZ from Retsö2009 to Retsö2009a.
2023-03-01 16:15:07,909 - Fetched TEI for B6L9AIMZ
2023-03-01 16:15:07,914 - Fetched TEI for N2NCTC8C
2023-03-01 16:15:07,919 - Fetched TEI for RCI9I9AM
2023-03-01 16:15:07,930 - Fetched TEI for 3H4YI6QY
2023-03-01 16:15:07,933 - Fetched TEI for E9GJGZZ7
2023-03-01 16:15:07,940 - Fetched TEI for QUCSQL79
2023-03-01 16:15:07,946 - Fetched TEI for EYRYK2V5
2023-03-